In [250]:
import pandas as pd
import re
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
import numpy as np

In [251]:
train = pd.read_csv("train_dataset_train.csv",index_col = 'id')
test = pd.read_csv("test_dataset_test.csv",index_col = 'id')

In [252]:
test = test.dropna()

In [253]:
def clear_text2(text):
    text=text.lower()
    text = re.findall('[а-я]+',text)
    return " ".join(text)

train['name'] = train['name'].apply(clear_text2)
test['name'] = test['name'].apply(clear_text2)

In [254]:
train

,name,groups
id,,
1034,шок цена пена д душа бритья куппер мл акс,10
1035,мин вода нагутская л,0
1036,пельмени домашние вес по прямицино,10
1037,печенье овсяно гречневое эко бот,10
1038,спред растительно жировой масляничка г с змж,10
...,...,...
659093,карачинская л шт мин вода,0
659094,хлеб донской г рузский х з,9
659095,печенье сахарное сэндвич с шоколадно сливочным...,6


In [255]:
X = train.iloc[:,:-1]
y = train.iloc[:,-1].values

In [256]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, test_size=0.0015, random_state=42)

In [257]:
pool_train = Pool(X_train, y_train, text_features = ['name'])
pool_test = Pool(X_test, y_test, text_features = ['name'])

In [262]:
#optuna
import optuna
def objective(trial):
   
    catboost_params = {
    'learning_rate':  trial.suggest_discrete_uniform("learning_rate", 0.02, 0.5, 0.02),
    'depth': trial.suggest_int("_depth", 1, 10),   
    'iterations': 3000,
    'random_state':1,    
    'task_type': 'GPU',
    'early_stopping_rounds': 10,
    'use_best_model': True,
    'verbose': False,
    'class_names': y_train}
    model = CatBoostClassifier(**catboost_params)
    model.fit(pool_train, eval_set = pool_test)
    
    y_pred = model.predict(X_eval,prediction_type = 'Class')
    test_pred = y_eval
    
    score = recall_score(test_pred, y_pred, average='macro')
    return score

study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials=10000, timeout=30000, show_progress_bar = True)

[I 2022-09-06 13:40:25,440] A new study created in memory with name: no-name-90389c16-ebf1-4e89-9dd7-6dfc8544f336
C:\Users\Yakov\anaconda3\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



  0%|          | 0/10000 [00:00<?, ?it/s]

Found only 9 unique classes in the data, but have defined 624222 classes. Probably something is wrong with data.


[I 2022-09-06 13:42:16,569] Trial 0 finished with value: 0.9497430112943616 and parameters: {'learning_rate': 0.28, '_depth': 5}. Best is trial 0 with value: 0.9497430112943616.


Found only 9 unique classes in the data, but have defined 624222 classes. Probably something is wrong with data.


...

[I 2022-09-06 21:56:39,944] Trial 237 finished with value: 0.9790639989486826 and parameters: {'learning_rate': 0.26, '_depth': 7}. Best is trial 41 with value: 0.981013316687474.


Found only 9 unique classes in the data, but have defined 624222 classes. Probably something is wrong with data.


[I 2022-09-06 21:58:31,004] Trial 238 finished with value: 0.9790639989486826 and parameters: {'learning_rate': 0.26, '_depth': 7}. Best is trial 41 with value: 0.981013316687474.


Found only 9 unique classes in the data, but have defined 624222 classes. Probably something is wrong with data.


[I 2022-09-06 22:00:53,977] Trial 239 finished with value: 0.9699607946207591 and parameters: {'learning_rate': 0.30000000000000004, '_depth': 9}. Best is trial 41 with value: 0.981013316687474.


In [263]:
study.best_value

0.981013316687474

In [264]:
optuna.visualization.plot_contour(study, params=['learning_rate','_depth'])

In [265]:
study.best_params

{'learning_rate': 0.26, '_depth': 9}

In [272]:
catboost_params = {
    'learning_rate':0.26,
    'depth': 9,
    'random_state':1,
    'iterations': 3000,
    'task_type': 'GPU',
    'early_stopping_rounds': 10,
    'use_best_model': True,
    'verbose': False,
    'class_names': y_train
}
model = CatBoostClassifier(**catboost_params)
model.fit(pool_train, eval_set = pool_test, plot=True)#, save_snapshot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Found only 9 unique classes in the data, but have defined 624222 classes. Probably something is wrong with data.


In [273]:
%%time
#y_pred = model.predict(X_test[0:20],prediction_type = 'Probability')
y_pred = model.predict(X_eval,prediction_type = 'Class')

test_pred = y_eval

CPU times: total: 55.2 s
Wall time: 42.9 s


In [274]:
#На локальном тестировании модель выдаёт такой результат
score = recall_score(test_pred, y_pred, average='macro')
print("Recall score:", score)

Recall score: 0.9717540574282147


In [267]:
%%time
test['groups'] = test['name'].map(lambda b: model.predict([b],prediction_type = 'Class')[0]) 

CPU times: total: 1d 13h 37s
Wall time: 1d 12h 51min 25s


In [268]:
test['groups'] = test['groups'].map(lambda b: str(b)+".0")

In [269]:
test['groups'].to_csv('sub.csv')

In [270]:
sub= pd.read_csv("sub.csv",index_col = 'id')

In [271]:
sub

,groups
id,
10529,10.0
10530,10.0
10531,10.0
10532,0.0
10533,10.0
...,...
292751,10.0
292752,7.0
292753,10.0


In [248]:
sub.to_csv("sub.csv")